# Apache Log Verileri ile Soru-Cevap Sistemi

Kullanılacak kütüphanelerin yüklenmesi

- pandas: Veri işleme ve analiz kütüphanesi
- sklearn: Makine öğrenimi kütüphanesi
- faiss: Vektör verileri için benzerlik arama kütüphanesi
- torch: PyTorch kütüphanesi
- transformers: Hugging Face tarafından geliştirilen model ve tokenizasyon kütüphanesi
- data_generator: Verileri oluşturmak için yazılan özel kütüphane

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from data_generator import DataGenerator

Veri Oluşturma
- data_generator kütüphanesi ile veri oluşturma
Not: Bu işlem  1 dakika 20 saniye civarında sürecektir. Hata alınması halinde 'data_generator/DataGenerator.py' dosyasını çalıştırınız.

In [2]:
DataGenerator.main()

Test dosyası başarıyla oluşturuldu!
Test dosyası oluşturuldu!
Log dosyası başarıyla CSV formatına dönüştürüldü!
Log dosyası başarıyla CSV formatına dönüştürüldü!


In [3]:
data = pd.read_csv('data/apache/data.csv')
print("Sütun Adları:", data.columns)

Sütun Adları: Index(['IP Address', 'Date', 'Request', 'Endpoint', 'Status Code',
       'Response Size', 'Referrer', 'User Agent', 'Time Taken'],
      dtype='object')


Veri Ön İşleme
- Tarih sütununun düzeltilmesi
- Eksik verilerin kontrolü
- Gereksiz sütunların çıkarılması
- Yıl, ay, gün, saat ve dakika sütunlarının oluşturulması
- Boş verilerin temizlenmesi
- Boş olmayan verilerin kontrolü
- Request, Endpoint ve Status Code sütunlarının birleştirilmesi

In [4]:
data['Date'] = pd.to_datetime(data['Date'].str.strip('[]'), format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')

missing_values = data.isnull().sum()
print("Eksik Veriler:", missing_values)

data = data.drop(columns=['Referrer', 'User Agent'], errors='ignore')

data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['Hour'] = data['Date'].dt.hour
data['Minute'] = data['Date'].dt.minute

data['Status Code'] = pd.to_numeric(data['Status Code'], errors='coerce')

data = data.dropna(subset=['Request', 'Endpoint', 'Status Code'])
data = data[data['Request'].str.strip() != '']
data = data[data['Endpoint'].str.strip() != '']
data = data[data['Status Code'].notna()] 

data['Combined'] = data['Request'] + ' ' + data['Endpoint'] + ' ' + data['Status Code'].astype(str)

print(data['Combined'].head())
print(data['Combined'].isnull().sum())


Eksik Veriler: IP Address       0
Date             0
Request          0
Endpoint         0
Status Code      0
Response Size    0
Referrer         0
User Agent       0
Time Taken       0
dtype: int64
0               PUT /usr/admin 200
1     PUT /usr/admin/developer 200
2     GET /usr/admin/developer 304
3    POST /usr/admin/developer 304
4     PUT /usr/admin/developer 200
Name: Combined, dtype: object
0


TF-IDF Vektörleri Oluşturma
- Tüm verilerin birleştirilmesi
- TF-IDF vektörlerinin oluşturulması
- Vektörlerin faiss kütüphanesi ile benzerlik araması için hazırlanması

In [5]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Combined']).toarray()

index = faiss.IndexFlatL2(X.shape[1])
index.add(X.astype('float32'))


Sorgu ve Yanıt Oluşturma
- Verilen sorguya en yakın logları bulma
- Bulunan loglar üzerinden model ile yanıt oluşturma
- Sorgu ve yanıtların test edilmesi
- Modelin yanıtlarının gösterilmesi

In [11]:
def find_relevant_logs(query):
    query_vector = vectorizer.transform([query]).toarray().astype('float32')
    D, I = index.search(query_vector, 5)
    return data.iloc[I[0]]

def generate_response(logs):
    input_text = " ".join(
           logs.apply(
        lambda row: f"{row['IP Address']} {row['Date']} {row['Request']} {row['Endpoint']} {row['Status Code']} "
                    f"{row['Response Size']}{row['Time Taken']}",
        axis=1
    ).tolist()
    )
    input_text = input_text[:1000] 

    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    output = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=50, num_return_sequences=1,
                            pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

def answer_question(query):
    relevant_logs = find_relevant_logs(query)
    response = generate_response(relevant_logs)
    return response

Model ve Token
- GPT-2 modeli ve tokenizasyonu yüklenmesi

In [12]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

C:\Users\reyiz\anaconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Test Sorguları Listesi
- Modelin test edilmesi için sorguların listesi

In [13]:
# Test Sorguları Listesi
queries = [
    "Son 24 saatte hangi URL'ler 500 hatası aldı?",
    "Son bir ayda hangi IP adresleri en fazla 403 hatası aldı?",
    "Son bir yıl içinde en sık kullanılan POST isteklerinin listesi nedir?",
    "Son 30 gün içinde hangi tarayıcılar en fazla 404 hatası aldı?",
    "Son haftada hangi endpoint'ler en yüksek Response Size'a sahipti?",
    "En son 10 istekte hangi User Agent'lar kullanıldı?",
    "En yüksek zaman alımı (Time Taken) olan 5 istek nedir?",
    "Son 6 ayda hangi Referrer en çok ziyaret edildi?",
    "Son 24 saatte hangi Endpoint'lerde 502 hatası alındı?",
    "Hangi IP adresleri en uzun süre GET isteği yaptı?",
    "Which IP adress has Longest GET time ?",
]

Her bir sorguyu test etme
- Her bir sorgu için modelin yanıtının gösterilmesi
- Sorgu ve yanıtların gösterilmesi

In [14]:
# Her bir sorguyu test etme
for query in queries:
    print("-" * 80)
    print("\n")
    
    response = answer_question(query)
    print(f"Sorgu: {query}")
    print(f"Modelin Yanıtı: {response}")
    print("\n")


--------------------------------------------------------------------------------


Sorgu: Son 24 saatte hangi URL'ler 500 hatası aldı?
Modelin Yanıtı: 65.98.111.139 2018-05-23 02:08:16+03:00 GET /usr 500 50821519 119.61.248.76 2018-07-10 02:58:33+03:00 GET /usr 500 49954689 175.246.90.41 2019-08-10 06:56:36+03:00 GET /usr 500 50282310 110.219.143.114 2018-09-07 05:52:47+03:00 GET /usr 500 50974713 138.65.120.154 2019-11-13 01:01:46+03:00 GET /usr 500 4976852 12.132.128.1 2018-09-07 05:52:47+03:00 GET /usr 500 4976852 12.132.128.1 2018-09-07 05:52:47+03:00 GET


--------------------------------------------------------------------------------


Sorgu: Son bir ayda hangi IP adresleri en fazla 403 hatası aldı?
Modelin Yanıtı: 36.247.110.174 2018-05-27 12:49:07+03:00 GET /usr 403 49224051 206.80.85.172 2018-06-12 11:09:40+03:00 GET /usr 403 50144954 111.117.166.35 2019-12-30 02:06:04+03:00 GET /usr 403 50361304 139.53.61.225 2018-09-30 10:00:41+03:00 GET /usr 403 49482547 216.109.15.188 201

Sorgular ile üretilen yanıtlar arasında bir ilişki olduğu gözlenmekte olup, modelin verileri doğru bir şekilde işlediği görülmektedir.